## Evaluate the Deformation of Bubbles of 2D Materials Using Energy-based PINN Scheme

### Basic Settings

In [ ]:
# Import necessary packages
import torch
import torch.nn as nn
from torch.optim import Adam, LBFGS
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
import pygmsh as pgm
import matplotlib
import matplotlib.pyplot as plt

# Select the running mode: True-training, False-prediction
runMode = False

# Choose the material
material = "graphene"

# Settle the calculation of my model onto GPU
torch.cuda.init()
if torch.cuda.is_available():
    torch.set_default_device("cuda")
    print("CUDA is being used")

### Parameters in the governing equations and normalization

Material constants are defined here, as well as the radius of the bubble and uniform pressure. To avoid possible ill-conditioned training caused by extreme values, the radius is normalized as
$$A=\frac{a}{h_\text{e}}$$
where $h_e$ is a pseudo-thickness expected to be specified by users to scale up geometric quantities. $h_\text{e}=0.1\text{nm}$ is set for reference.

The examples presented cover four types of symmetries of 2D crystals, including
- Graphene(hexagonal) referred in https://doi.org/10.1115/1.4024169
- $\text{Mn}_2\text{S}_2$ (square) referred in https://c2db.fysik.dtu.dk/material/2MnS-2
- Black phosphorene (BP, rectangular) referred in https://doi.org/10.1063/1.4885215
- $\text{PdCdCl}_4$ (oblique) referred in https://c2db.fysik.dtu.dk/material/1CdPdCl4-1

The material non-linearity of graphene can be referred in http://dx.doi.org/10.1103/PhysRevB.80.205407.



In [ ]:
a = 10e-9   # radius
q = 307.4e6 # transverse loading

a = 2*a
q = 3*q

# Material constants [SI units]

'''
Examples of material constants [SI units]
-----------------------------------------
Hexagonal: Graphene
Input parameters:
C11_2D = 354.1
C22_2D = 354.1
C12_2D = 56.7
C16_2D = 0
C26_2D = 0
C66_2D = 148.7
-----------------------------------------
Square: Mn2S2
Input parameters:
C11_2D = 121.83
C22_2D = 121.83
C12_2D = 33.90
C16_2D = 0
C26_2D = 0
C66_2D = 108.45
-----------------------------------------
Rectangular: BP
Input parameters:
C11_2D = 102.98
C22_2D = 27.30
C12_2D = 17.51
C16_2D = 0
C26_2D = 0
C66_2D = 22.76
-----------------------------------------
Oblique: PdCdCl4
Input parameters:
C11_2D = 12.38
C22_2D = 37.00
C12_2D = 8.50
C16_2D = 3.24
C26_2D = 9.76
C66_2D = 14.48
'''

# Material constants
C11_2D = 354.1
C22_2D = 354.1
C12_2D = 56.7
C16_2D = 0
C26_2D = 0
C66_2D = 148.7

# Effective thickness
he = 0.1e-9

A = a/he

### Define the neural network

We have three outputs, $U$, $V$ and $W$ in total, and we assign each of them with an individual network to improve the efficiency and accuracy.

These outputs don't correspond to the real displacement of the bubble, but the normalized ones
$$U=\frac{u}{h_\text{e}}, V = \frac{v}{h_\text{e}}, W = \frac{w}{h_\text{e}}$$
Similarly, the inputs $X$, $Y$ relates to the real coordinates through
$$X = \frac{x}{h_\text{e}}, Y = \frac{y}{h_\text{e}}$$



In [ ]:
class Net(nn.Module):
    # Define a new class inheriting nn.Module
    def __init__(self, nInput, nOutput, hiddenSizes):
        # Initialize basic information for the network
        super(Net, self).__init__()

        # Define the input layer: reflection + weight + bias
        self.Input = nn.Linear(nInput, hiddenSizes[0])
        nn.init.xavier_uniform_(self.Input.weight)
        nn.init.normal_(self.Input.bias)

        # Define the output layer: reflection + weight + bias
        self.Output = nn.Linear(hiddenSizes[-1], nOutput)
        nn.init.xavier_uniform_(self.Output.weight)
        nn.init.normal_(self.Output.bias)

        # Define the hidden layers: reflection + weight + bias
        self.Hidden = nn.ModuleList()
        for i in range(len(hiddenSizes) - 1):
            layer = nn.Linear(hiddenSizes[i], hiddenSizes[i+1])
            nn.init.xavier_uniform_(layer.weight)
            nn.init.normal_(layer.bias)
            self.Hidden.append(layer)

    # Define the activation function
    def forward(self, inputData):
        tensor = torch.tanh(self.Input(inputData))
        for layer in self.Hidden:
            tensor = torch.tanh(layer(tensor))
        outputData = self.Output(tensor)
        return outputData

### Define the derivatives

In [ ]:
def Derivatives(inputData, Net, func_HBC):
    outputData = Net(inputData)*(func_HBC(inputData).view(-1, 1))
    # Evaluate first derivatives
    jacobian = torch.autograd.grad(outputData,
                                   inputData,
                                   torch.ones_like(outputData),
                                   retain_graph = True,
                                   create_graph = True,
                                   allow_unused = True)
    partial_X = jacobian[0][:, 0].view(-1, 1)
    partial_Y = jacobian[0][:, 1].view(-1, 1)

    return partial_X, partial_Y

### Evaluate the free energy
This is the cell where users can define constitutive laws.

Due to the normalization for both coordinates and displacements, the normalized strains are equal to the real strain, which is shown as followings
$$E_{ij} = \frac{1}{2}(\partial_{X_i}U_j + \partial_{X_j}U_i + \partial_{X_i}W\partial_{X_j}W)=\varepsilon_{ij}$$
The membrane forces can be determined in terms of strains once the constitutive relation is known.

In [ ]:
def freeEnergy(inputData, NetU, NetV, NetW, funcHBC):
    # Deflection
    W = NetW(inputData)*(funcHBC(inputData).view(-1, 1))

    # Evaluate derivatives
    U_X, U_Y = Derivatives(inputData, NetU, funcHBC)
    V_X, V_Y = Derivatives(inputData, NetV, funcHBC)
    W_X, W_Y = Derivatives(inputData, NetW, funcHBC)

    # Evaluate strains and curvatures
    EX = U_X + 0.5*W_X**2
    EY = V_Y + 0.5*W_Y**2
    GXY = U_Y + V_X + W_X*W_Y

    # Constitutive law
    NX = C11_2D*EX + C12_2D*EY + C16_2D*GXY
    NY = C12_2D*EX + C22_2D*EY + C26_2D*GXY
    NXY = C16_2D*EX + C26_2D*EY + C66_2D*GXY

    # Evaluate potential energy density
    U_m = 0.5*(NX*EX + NY*EY + NXY*GXY)

    '''
    The material non-linearity for graphene
    U_m_nl = (1/6*(C111_2D*EX**3 + C222_2D*EY**3) + C112_2D*EX**2*EY +
              (C111_2D - C222_2D + C112_2D)*EX*EY**2 + (3/4*C222_2D - 1/2*C111_2D - 1/4*C112_2D)*EX*GXY**2 +
              (1/2*C111_2D - 1/4*C222_2D - 1/4*C112_2D)*EY*GXY**2)
    U_m = U_m + U_m_nl
    '''

    # Evaluate the total energy over the domain
    Phi_m = torch.mean(U_m)
    V = torch.mean(-q*W*he)

    return Phi_m, V

### Define the training data

To ensure training points are uniformly distributed over the domain,
$$R=A\sqrt{U_1},\theta=2\pi U_2$$
where $U_1 \sim  \text{Uniform}(0,1)$ and $U_2 \sim \text{Uniform}(0, 1)$ are independent.

In [ ]:
def trainingData(nSample):
    R = A*torch.sqrt(torch.rand(nSample))
    Theta = 2*torch.pi*torch.rand(nSample)
    X, Y = R*torch.cos(Theta), R*torch.sin(Theta)
    coordinates = (torch.stack((X, Y), dim=1).requires_grad_(True))

    return coordinates

### Preparation before training
Auxiliary function $(1-r^2/A^2)$ is used to guarantee the fixed boundary conditions.

In [ ]:
# Set up training data
nSample = 100000
inputData = trainingData(nSample)

# Construct neural networks for each output
hiddenSizesW = [32, 64, 64, 64, 32]
hiddenSizesUV = [32, 64, 64, 64, 32]

NetU = Net(2, 1, hiddenSizesUV)
NetV = Net(2, 1, hiddenSizesUV)
NetW = Net(2, 1, hiddenSizesW)

# Impose hard boundary conditions
def funcHBC(inputData):
    R_square = inputData[:, 0]**2 + inputData[:, 1]**2

    return (1 - R_square/A**2)


### Training
Optimize the training using Adam with a learning rate scheduler.

In [ ]:
# Set up parameters for outer training
nEpochesAdam = 20000 # 15000
intervalAdam = 100
learningRate = 2e-3

# Set up details for optimizers
optimizerAdam = Adam(
    list(NetU.parameters()) + list(NetV.parameters()) + list(NetW.parameters()) ,
    lr=learningRate,
    weight_decay=0)

scheduler = ReduceLROnPlateau(
    optimizerAdam,
    mode='min',
    factor=0.5,
    patience=2000,
    threshold=1e-4,
    min_lr=1e-5
)

# Set up recorders for monitoring
lossHistory_adam = []
energy = {'membrane': None, "external": None}

if runMode == True:
    # First stage: Adam
    #print('First stage: Adam')
    for epoch in range(nEpochesAdam):
        # Evaluate the loss then implement backpropagation
        Phi_m, V = freeEnergy(inputData, NetU, NetV, NetW, funcHBC)
        loss = Phi_m + V
        loss.backward()
        optimizerAdam.step()
        optimizerAdam.zero_grad()
        scheduler.step(loss.item())

        # Monitor
        if (epoch + 1) % intervalAdam == 0:
            # Re-sampling
            inputData = trainingData(nSample)

            # Record the loss history
            loss_Phi_m = Phi_m.detach().cpu().item()
            loss_V = V.detach().cpu().item()
            totalLoss = loss_Phi_m + loss_V
            lossHistory_adam.append([totalLoss, loss_Phi_m, loss_V])
            print(f'Epoch:{epoch+1}, '
                  f'Total:{totalLoss:.4e}, '
                  f'Membrane:{loss_Phi_m:.4e}, '
                  f'External:{loss_V:.4e} ')

    # Show the loss history
    timeAdam = [(i + 1)*intervalAdam for i in range(len(lossHistory_adam))]
    lossHistory_adam = np.array(lossHistory_adam)

    loss_data = np.column_stack((timeAdam, lossHistory_adam))
    np.savetxt(f'{material}-membrane-loss.dat', loss_data, fmt='%.8f')

    fig = plt.figure()

    # Total loss
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(timeAdam, lossHistory_adam[:, 0], label='Total', color='blue')
    ax1.set_xlabel('Epoches')
    ax1.set_ylabel('Loss(energy)')
    ax1.grid(True)
    ax1.legend(loc='best')

    # Energy in time
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(timeAdam, lossHistory_adam[:, 1], label='Membrane', color='red')
    ax2.plot(timeAdam, lossHistory_adam[:, 2], label='External', color='orange')
    ax2.set_xlabel('Epoches')
    ax2.set_ylabel('Loss')
    ax2.legend(loc='best')
    ax2.grid(True)

    plt.tight_layout()
    plt.show()


    # Save the model
    torch.save(NetU.state_dict(), f'{material}-membrane-NetU.pth')
    torch.save(NetV.state_dict(), f'{material}-membrane-NetV.pth')
    torch.save(NetW.state_dict(), f'{material}-membrane-NetW.pth')


### Prediction

In [ ]:
if runMode == False:
    # Load the model's parameters
    NetU.load_state_dict(torch.load(f'{material}-membrane-NetU.pth', weights_only=True))
    NetV.load_state_dict(torch.load(f'{material}-membrane-NetV.pth', weights_only=True))
    NetW.load_state_dict(torch.load(f'{material}-membrane-NetW.pth', weights_only=True))

    # Set the model to evaluation mode (important for inference)
    NetU.eval()
    NetV.eval()
    NetW.eval()

meshSize = A/200
with pgm.geo.Geometry() as geom:
    disk = geom.add_circle(x0=[0.0, 0.0, 0.0], radius=A, mesh_size=meshSize)
    diskMesh = geom.generate_mesh()
    coordinates = torch.tensor(diskMesh.points[:, 0:2],
                               dtype=torch.float32,
                               requires_grad=True)

# Prediction
U = NetU(coordinates)*(funcHBC(coordinates).view(-1, 1))
V = NetV(coordinates)*(funcHBC(coordinates).view(-1, 1))
W = NetW(coordinates)*(funcHBC(coordinates).view(-1, 1))

# Transport all the tensors onto CPU for post-process(nanometer)
x = coordinates[:, 0].detach().cpu().numpy().reshape(-1)*he*1e9
y = coordinates[:, 1].detach().cpu().numpy().reshape(-1)*he*1e9
r = np.linspace(-A, A, 200)*he*1e9
u = U.detach().cpu().numpy().reshape(-1)*he*1e9
v = V.detach().cpu().numpy().reshape(-1)*he*1e9
w = W.detach().cpu().numpy().reshape(-1)*he*1e9

# Check the maximum deflection
print(f'The maximum deflection is {max(w)}nm')

# Save the data from prediction
predictData = np.stack((x, y, u, v, w), axis=1)
np.savetxt(fname=f'{material}-membrane-prediction.dat',X=predictData,fmt='%.8f')


### Visualization

In [ ]:
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Surface
fig1 = plt.figure(figsize=(18, 6))

subplotw = fig1.add_subplot(131, projection='3d')
surfw = subplotw.plot_trisurf(x, y, w, cmap='viridis', linewidth=0.2, antialiased=True)
subplotw.set_title(r'Deflection w', fontsize=14)
subplotw.set_xlabel(r'x/nm', fontsize=12)
subplotw.set_ylabel(r'y/nm', fontsize=12)
subplotw.set_zlabel(r'w/nm', fontsize=12)
fig1.colorbar(surfw, ax=subplotw, shrink=0.5, aspect=8, fraction=0.15, pad=0.1)

subplotu = fig1.add_subplot(132, projection='3d')
surfu = subplotu.plot_trisurf(x, y, u, cmap='viridis', linewidth=0.2, antialiased=True)
subplotu.set_title(r'In-plane Displacement u', fontsize=14)
subplotu.set_xlabel(r'x/nm', fontsize=12)
subplotu.set_ylabel(r'y/nm', fontsize=12)
subplotu.set_zlabel(r'u/nm', fontsize=12)
fig1.colorbar(surfu, ax=subplotu, shrink=0.5, aspect=8, fraction=0.15, pad=0.1)

subplotv = fig1.add_subplot(133, projection='3d')
surfv = subplotv.plot_trisurf(x, y, v, cmap='viridis', linewidth=0.2, antialiased=True)
subplotv.set_title(r'In-plane Displacement v', fontsize=14)
subplotv.set_xlabel(r'x/nm', fontsize=12)
subplotv.set_ylabel(r'y/nm', fontsize=12)
subplotv.set_zlabel(r'v/nm', fontsize=12)
fig1.colorbar(surfv, ax=subplotv, shrink=0.5, aspect=8, fraction=0.15, pad=0.1)

plt.tight_layout()
plt.show()

# Scatter
fig2 = plt.figure(figsize=(18, 6))

subplotww = fig2.add_subplot(131)
scw = subplotww.scatter(x, y, c=w, cmap='rainbow')
subplotww.set_title(r'Deflection w/nm', fontsize=16)
subplotww.set_xlabel(r'x/nm', fontsize=14)
subplotww.set_ylabel(r'y/nm', fontsize=14)
subplotww.set_aspect('equal')
cbw = plt.colorbar(scw, ax=subplotww, shrink=0.8, aspect=10)
cbw.set_label('Magnitude', fontsize=12)

subplotuu = fig2.add_subplot(132)
scu = subplotuu.scatter(x, y, c=u, cmap='rainbow')
subplotuu.set_title(r'In-plane Displacement u/nm', fontsize=16)
subplotuu.set_xlabel(r'x/nm', fontsize=14)
subplotuu.set_ylabel(r'y/nm', fontsize=14)
subplotuu.set_aspect('equal')
cbu = plt.colorbar(scu, ax=subplotuu, shrink=0.8, aspect=10)
cbu.set_label('Magnitude', fontsize=12)

subplotvv = fig2.add_subplot(133)
scv = subplotvv.scatter(x, y, c=v, cmap='rainbow')
subplotvv.set_title(r'In-plane Displacement v/nm', fontsize=16)
subplotvv.set_xlabel(r'x/nm', fontsize=14)
subplotvv.set_ylabel(r'y/nm', fontsize=14)
subplotvv.set_aspect('equal')
cbv = plt.colorbar(scv, ax=subplotvv, shrink=0.8, aspect=10)
cbv.set_label('Magnitude', fontsize=12)

plt.tight_layout()
plt.show()